In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import math
from datetime import timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from matplotlib.colors import LogNorm
import matplotlib.ticker as mticker
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.ticker import FuncFormatter


In [ ]:
df_raw = pd.read_csv('raw.csv')
df_raw.columns = df_raw.columns.str.lower()
print("shape of the data:", df_raw.shape)
df_raw.head()

# 1. Data cleaning

In [ ]:
# check data types of all columns
df_raw.dtypes

In [ ]:
def convert_to_datetime(df, columns, errors='coerce', format=None):
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors=errors, format=format)
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")
    return df

In [ ]:
df_raw = convert_to_datetime(df_raw, ['timestamp', 'converted_at', 'trial_start', 'trial_end'])
df_raw['converted'] = df_raw['converted'].astype(int)
print(df_raw.dtypes)
df_raw.head()

In [ ]:
df_raw.isnull().sum()

In [ ]:
# Checking if 'converted_at' is null is only null when converted=False

df_nulls = df_raw[df_raw['converted_at'].isna()]
null_summary = df_nulls.groupby('converted').size().reset_index(name='null_count')
null_summary

Since organizations that did not convert churned by the end of trial, I will consider their this timestamp as their 'conversion into churn' time.

In [ ]:
# Filling converted_at for churned users.

#latest_timestamps = df_raw.groupby('organization_id')['timestamp'].transform('max')

df_raw['converted_at'] = df_raw.apply(
    lambda row: row['trial_end'] if pd.isna(row['converted_at']) and not row['converted'] else row['converted_at'],
    axis=1
)

df_raw = df_raw.sort_values(by=['organization_id', 'timestamp']).reset_index(drop=True)
df_raw

For this analysis I need only the activities that happenned before conversion, so I will remove the rest

In [ ]:
#df_bc = df[~(df['timestamp'] > df['converted_at'])]
#df_bc.reset_index(drop=True)

# 2. Finding and cleaning inconsistencies

- Converted value per organization should remain unchanged
- All the timestamps for not converted organizations must be within the trial period

In [ ]:
def check_conversion_consistency(df):
    
    converted_counts = df.groupby('organization_id')['converted'].nunique()
    # Find the organizations that have more than 1 distinct conversion rate
    orgs_with_multiple_converted = converted_counts[converted_counts > 1].index.tolist()

    if orgs_with_multiple_converted:
        return orgs_with_multiple_converted
    else:
        return "No inconsistent conversion values"

In [ ]:
result = check_conversion_consistency(df_raw)
print(result)

In [ ]:
def check_date_consistency(df, return_message=True):

    # === Aggregate only necessary columns ===
    agg_dict = {
        'timestamp': ['min', 'max'],
        'converted_at': ['min', 'max'],
        'trial_start': ['min', 'max'],
        'trial_end': ['min', 'max']
    }

    date_check = df.groupby('organization_id').agg(agg_dict)
    date_check.columns = ['_'.join(col).strip() for col in date_check.columns.values]
    date_check = date_check.reset_index()

    # === Making sure that converted_at, trial_start and trial_end are unique per onganization ===
    for col in ['converted_at', 'trial_start', 'trial_end']:
        date_check[f"{col}_vary_flag"] = (
            date_check[f"{col}_min"] != date_check[f"{col}_max"]
        ).astype(int)

    # === Checking that activity timestamps are within allowed ranges ===
    date_check['timestamp_min_flag'] = (
        date_check['timestamp_min'] <= date_check['trial_start_min']
    ).astype(int)

    date_check['timestamp_max_flag'] = (
        date_check['timestamp_max'] >= date_check['trial_end_max']
    ).astype(int)

    date_check['converted_at_flag'] = (
        date_check['converted_at_min'] < date_check['trial_start_min']
    ).astype(int)

    # === Identifying suspicious records ===
    flag_cols = [col for col in date_check.columns if col.endswith('_flag')]
    suspicious = date_check[date_check[flag_cols].sum(axis=1) > 0]

    if suspicious.empty:
        return "No inconsistent dates" if return_message else None

    return suspicious

In [ ]:
result = check_date_consistency(df_raw)
print(result)

# 3. Checking for duplicates

Checking for duplicated events using a combination of **organization_id**, **activity_name** and **timestamp** as a key

In [ ]:
grouped_records = (
    df_raw
    .groupby(['organization_id', 'activity_name', 'timestamp'])
    .agg(
        total_records=('activity_name', 'size')
    )
    .sort_values(['organization_id','timestamp', 'activity_name'])
    .reset_index()
)

In [ ]:
num_orgs_total = df_raw['organization_id'].nunique()
num_rows_total = len(df_raw)

duplicates = grouped_records[grouped_records['total_records'] > 1]

num_orgs_with_duplicates = duplicates['organization_id'].nunique()
num_duplicated_rows = duplicates['total_records'].sum()

print("Total unique organizations:", num_orgs_total)
print("Total records:", num_rows_total)

print("Number of organizations with duplicates:", num_orgs_with_duplicates)
print("Total number of duplicated records:", num_duplicated_rows)

In [ ]:
# Checking the number of duplicates per activity

duplicates_activity = (
    duplicates
    .groupby('activity_name')
    .agg(
        total_duplicate_instances=('total_records', 'size'),
        total_orgs=('organization_id', 'nunique'),
        min_total_records=('total_records', 'min'),
        percentile_25=('total_records', lambda x: x.quantile(0.25)),
        median_total_records=('total_records', 'median'),
        percentile_75=('total_records', lambda x: x.quantile(0.75)),
        max_total_records=('total_records', 'max')
    )
    .sort_values('median_total_records', ascending=False)
    .reset_index()
)

duplicates_activity

#### Observations:

Looking at the median total records, I can see that most activities have 2-3 duplicates per organization and timestamp, so that could be a simple coincindece. 

However, looking at max, there are 3 activities that stand out: 

- Scheduling.Shift.Created
- Scheduling.Shift.AssignmentChanged
- Scheduling.Availability.Set

I assume that these activities can be triggered in bulk. For example, an Employee can set availability for several days with one click, and an Admin can create several shifts using some presets or a template. 

I'm going to verify whether they are indeed triggered by something else.

In [ ]:
#Identifying records with max duplicates per activity

idx = grouped_records.groupby('activity_name')['total_records'].idxmax()
max_per_act = grouped_records.loc[idx]
max_per_act = max_per_act.sort_values('total_records', ascending=False).reset_index(drop=True)
max_per_act.head(10)

In [ ]:
# Making sure that in case of timestamp conflict the template is shown 

gr_sorted = grouped_records.sort_values(
    by=['organization_id', 'timestamp', 'activity_name'],
    ascending=[True, True, True]
)

# Define custom sorting priority
activity_priority = {
    'Scheduling.Template.ApplyModal.Applied': 0,
    'Scheduling.Shift.Created': 1,
    'Scheduling.Shift.AssignmentChanged': 2
}

# Apply priority where applicable
gr_sorted['activity_priority'] = gr_sorted['activity_name'].map(activity_priority).fillna(999)

# Final sort with priority
gr_sorted = gr_sorted.sort_values(
    by=['organization_id', 'timestamp', 'activity_priority']
).drop(columns='activity_priority').reset_index(drop=True)

#gr_sorted

In [ ]:
def get_records_within_interval(
    source_df: pd.DataFrame,
    index: int,
    search_df: pd.DataFrame,
    time_before: int = None,
    time_after: int = None
) -> pd.DataFrame:

    # Extract reference values
    org_id = source_df.loc[index, 'organization_id']
    ref_time = source_df.loc[index, 'timestamp']

    # Ensure timestamp column is datetime
    if not pd.api.types.is_datetime64_any_dtype(search_df['timestamp']):
        search_df = search_df.copy()
        search_df['timestamp'] = pd.to_datetime(search_df['timestamp'])

    # Start with org_id match
    mask = (search_df['organization_id'] == org_id)

    # Apply optional time bounds
    if time_before is not None:
        start_time = ref_time - timedelta(minutes=time_before)
        mask &= (search_df['timestamp'] >= start_time)

    if time_after is not None:
        end_time = ref_time + timedelta(minutes=time_after)
        mask &= (search_df['timestamp'] <= end_time)

    return search_df[mask]

In [ ]:
# Get records for index 0 of max_record_per_org within 60 minutes
result = get_records_within_interval(
    source_df=max_per_act,
    index=1,
    time_before=60,
    time_after=60,
    search_df=gr_sorted
)

result.reset_index(drop=True)

In [ ]:
def annotate_action_ever_used(
    source_df: pd.DataFrame,
    action_name: str = "Scheduling.Template.ApplyModal.Applied",
    max_records: int = 10
) -> pd.DataFrame:

    df = source_df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['organization_id', 'timestamp'])

    # Get first time the action was used per organization
    action_first_use = (
        df[df['activity_name'] == action_name]
        .groupby('organization_id')['timestamp']
        .min()
        .rename('first_template_used')
        .reset_index()
    )

    # Merge with original data
    df = df.merge(action_first_use, on='organization_id', how='left')

    # template_used_ever = 1 if first_template_used exists and is earlier than current timestamp
    df['template_used_ever'] = (
        (df['first_template_used'].notna()) &
        (df['first_template_used'] <= df['timestamp'])
    ).astype(int)

    # Drop helper column
    df = df.drop(columns='first_template_used')

    # Filter if desired
    df = df[df['total_records'] >= max_records].reset_index(drop=True)

    return df

In [ ]:
annotated_df = annotate_action_ever_used(
    source_df=gr_sorted,
    action_name="Scheduling.Template.ApplyModal.Applied",
    #time_interval=15,     # Optional: only recent activity
    max_records=10        # Only annotate rows where total_records >= 10
)

annotated_df

In [ ]:
# Filter rows where neither recent nor ever action occurred
no_template_triggered = annotated_df[
    (annotated_df['template_used_ever'] == 0)
]

pd.set_option('display.max_rows', 50)
#no_template_triggered.head(10)

# Find the index of the row with max total_records per organization
idx = no_template_triggered.groupby('organization_id')['total_records'].idxmax()
max_records_no_template = no_template_triggered.loc[idx].reset_index(drop=True)
max_records_no_template.sort_values('total_records', ascending=False).reset_index(drop=True)

In [ ]:
num_orgs = no_template_triggered['organization_id'].nunique()
num_rows = no_template_triggered['total_records'].sum()

print("Total unique organizations:", num_orgs)
print("Total records:", num_rows)

In [ ]:
# Filter rows for a specific record
check = gr_sorted[
    (gr_sorted['organization_id'] == '154647fa6ad39ad1ea4dd6bdfd273679') &
    (gr_sorted['timestamp'] >= pd.to_datetime('2024-02-01 00:00:00'))&
    (gr_sorted['timestamp'] <= pd.to_datetime('2024-02-22 20:00:00'))
].reset_index(drop=True)

pd.set_option('display.max_rows', 20)

check

#### Observations:

- Scheduling.Shift.Created and Scheduling.Shift.AssignmentChanged- many bulk records are indeed triggered by Scheduling.Template.ApplyModal.Applied
- Some duplicated entries are created by organizations that never used the templates
- Scheduling.Availability.Set - this is not, so I assume an employee can just submit their availability for the whole year with one click.

I will assume that all the duplicated records are valid and keep them.

In [ ]:
#Condesging the dataset for easier readability
activity_priority = {
    'Scheduling.Template.ApplyModal.Applied': 0,
    'Scheduling.Shift.Created': 1,
    'Scheduling.Shift.AssignmentChanged': 2
}

df_grp = (
    df_raw
    .groupby(['organization_id', 'activity_name', 'timestamp'], as_index=False)
    .agg(
        records=('activity_name', 'size'),
        converted=('converted', 'first'),
        converted_at=('converted_at', 'first'),
        trial_start=('trial_start', 'first'),
        trial_end=('trial_end', 'first')
    )
)

df_grp['activity_priority'] = df_grp['activity_name'].map(activity_priority).fillna(999)
df_grp = df_grp.sort_values(['organization_id', 'timestamp', 'activity_priority']).reset_index(drop=True)
df_grp = df_grp.drop(columns='activity_priority')
df_grp['bulk'] = (df_grp['records'] > 1).astype(int)

df_grp

In [ ]:
df_grp_sorted = df_grp.sort_values(['organization_id', 'timestamp']).reset_index(drop=True)

# Creating group_id for consecutive runs of same org/activity_name/bulk
df_grp_sorted['group_id'] = (
    (df_grp_sorted['organization_id'] != df_grp_sorted['organization_id'].shift()) |
    (df_grp_sorted['activity_name'] != df_grp_sorted['activity_name'].shift()) |
    (df_grp_sorted['bulk'] != df_grp_sorted['bulk'].shift()) |
    (df_grp_sorted['timestamp'].dt.date != df_grp_sorted['timestamp'].dt.date.shift())
).cumsum()

df_short = (
    df_grp_sorted
    .groupby('group_id')
    .agg(
        organization_id=('organization_id', 'first'),
        activity_name=('activity_name', 'first'),
        bulk=('bulk', 'first'),
        ts_start=('timestamp', 'first'),
        ts_end=('timestamp', 'last'),
        events=('records', 'size'), # number of rows condensed
        records=('records', 'sum'), # number of records in rows condensed
        converted=('converted', 'first'),
        converted_at=('converted_at', 'first'),
        trial_start=('trial_start', 'first'),
        trial_end=('trial_end', 'first')
    )
    .reset_index(drop=True)
)
df_short['time_diff_sec'] = (df_short['ts_end']-df_short['ts_start']).dt.total_seconds().astype(int)
df_short = df_short[['organization_id', 'activity_name', 'ts_start', 'ts_end', 'time_diff_sec', 'bulk', 'events', 'records', 'converted', 'converted_at', 'trial_start','trial_end']]
df_short

# 4. EDA

In [ ]:
df_eda = df_short.copy()
df_eda

#### Adding helper columns

In [ ]:
# Dimension columns
df_eda['first_action_ts'] = df_eda.groupby('organization_id')['ts_start'].transform('min') #first action timestamp
df_eda['first_action_weekday'] = df_eda['first_action_ts'].dt.dayofweek
df_eda['first_action_hour'] = df_eda['first_action_ts'].dt.hour
df_eda['last_action_ts'] = df_eda.groupby('organization_id')['ts_end'].transform('max')
df_eda['days_to_action'] = (df_eda['first_action_ts'].dt.date - df_eda['trial_start'].dt.date).dt.days
df_eda['active_span_days'] = ((df_eda['last_action_ts'] - df_eda['first_action_ts']).dt.days)+1

#Fact columns
df_eda['days_since_first_action'] = (df_eda['ts_start'] - df_eda['first_action_ts']).dt.days
df_eda['days_since_trial_start'] = (df_eda['ts_start'].dt.date - df_eda['trial_start'].dt.date).dt.days
df_eda['days_till_trial_end'] = (df_eda['trial_end'].dt.date - df_eda['ts_end'].dt.date).dt.days

df_eda

In [ ]:
# When do users convert?

df_eda['days_to_convert_tr'] = (df_eda['converted_at'].dt.date - df_eda['trial_start'].dt.date).dt.days #since trial start
df_eda['days_to_convert_fa'] = (df_eda['converted_at'].dt.date - df_eda['first_action_ts'].dt.date).dt.days #since first action
df_eda['days_to_convert_la'] = (df_eda['converted_at'].dt.date - df_eda['last_action_ts'].dt.date).dt.days #since first action

#Adding user group based on conversion status an conversion time
df_eda['user_group'] = np.select(
    [
        df_eda['converted'] == 0,  # Not converted
        (df_eda['converted'] == 1) & 
        (df_eda['converted_at'] <= df_eda['trial_end']) & 
        (df_eda['converted_at'] <= df_eda['last_action_ts']),  # Converted during trial & active after
        (df_eda['converted'] == 1) & 
        (df_eda['converted_at'] <= df_eda['trial_end']) & 
        (df_eda['converted_at'] > df_eda['last_action_ts']),   # Converted during trial & not active after
        (df_eda['converted'] == 1) & 
        (df_eda['converted_at'] > df_eda['trial_end']),  # Converted after trial
    ],
    [0, 1, 2, 3],  # Custom group labels
    default=4  # Catch-all for invalid/missing cases
)

#checking if all the organization fall into valud groups (not 3)
orgs_in_groups = (
    df_eda
    .groupby(['user_group'])
    .agg(organization_count=('organization_id', 'nunique'))
    .sort_index()
    .reset_index()
)

orgs_in_groups

In [ ]:
# Extract date and week info from ts_start
df_eda['activity_date'] = df_eda['ts_start'].dt.date
df_eda['activity_week'] = df_eda['ts_start'].dt.to_period('W').astype(str)

# Count distinct days per organization
active_days_per_org = (
    df_eda
    .groupby('organization_id')['activity_date']
    .nunique()
    .rename('active_days')
)

# Count distinct weeks per organization
active_weeks_per_org = (
    df_eda
    .groupby('organization_id')['activity_week']
    .nunique()
    .rename('active_weeks')
)

df_eda = df_eda.merge(active_days_per_org, on='organization_id', how='left')
df_eda = df_eda.merge(active_weeks_per_org, on='organization_id', how='left')

df_eda.drop(columns=['activity_date', 'activity_week'], inplace=True)

df_eda['activity_density'] = df_eda['active_weeks']/df_eda['active_days']

In [ ]:
df_eda

#### Exploratory Analysis

In [ ]:
# Visualizing convesion per days to converst since first action

converters = (
    df_eda[df_eda['converted'] == 1]
    .groupby('organization_id')
    .agg(days_to_convert=('days_to_convert_tr', 'min')) #change here days_to_convert_tr to days_to_convert_fa to check that parameter
    .sort_index()
    .reset_index()
)

# Group by days_to_convert
time_to_convert = (
    converters
    .groupby('days_to_convert')
    .agg(total_orgs=('organization_id', 'nunique'))
    .sort_index()
    .reset_index()
)

# Add cumulative converters and % share
time_to_convert['cumulative_converters'] = time_to_convert['total_orgs'].cumsum()
total_converted = time_to_convert['total_orgs'].sum()
time_to_convert['cumulative_percent'] = (time_to_convert['cumulative_converters'] / total_converted * 100).round(1)

# Create figure with secondary y-axis
fig = go.Figure()

# Add absolute converters (left y-axis)
fig.add_trace(
    go.Bar(
        x=time_to_convert['days_to_convert'],
        y=time_to_convert['total_orgs'],
        name='Daily Converters',
        marker_color='steelblue',
        yaxis='y1'
    )
)

# Add cumulative % converters (right y-axis)
fig.add_trace(
    go.Scatter(
        x=time_to_convert['days_to_convert'],
        y=time_to_convert['cumulative_percent'],
        name='Cumulative %',
        mode='lines+markers',
        marker_color='orange',
        yaxis='y2'
    )
)

# Update layout with dual axes
fig.update_layout(
    title='Daily and Cumulative Conversion by Days Since Trial Start',
    xaxis=dict(title='Trial Day', dtick=1),
    yaxis=dict(
        title='Number of Converted Orgs',
        side='left',
        showgrid=False
    ),
    yaxis2=dict(
        title='Cumulative % of Conversions',
        overlaying='y',
        side='right',
        range=[0, 100]
    ),
    legend=dict(x=0.01, y=0.99),
    width=900,
    height=500
)

fig.show()

In [ ]:
df_eda['trial_start_week'] = df_eda['trial_start'].dt.to_period('W').apply(lambda r: r.start_time).dt.normalize()
df_eda['conversion_week'] = df_eda['converted_at'].dt.to_period('W').apply(lambda r: r.start_time).dt.normalize()

df_eda['weeks_since_trial_start'] = ((df_eda['conversion_week'] - df_eda['trial_start_week']).dt.days // 7).clip(lower=0)

converted = df_eda[df_eda['converted'] == 1].copy()

conversion_counts = (
    converted.groupby(['trial_start_week', 'weeks_since_trial_start'])
    .agg(converted_orgs=('organization_id', 'nunique'))
    .reset_index()
)

cohort_sizes = (
    df_eda.groupby('trial_start_week')
    .agg(total_orgs=('organization_id', 'nunique'))
    .reset_index()
)

conversion_counts = conversion_counts.merge(cohort_sizes, on='trial_start_week')
conversion_counts['conversion_rate'] = conversion_counts['converted_orgs'] / conversion_counts['total_orgs']

heatmap_data = conversion_counts.pivot(index='trial_start_week', columns='weeks_since_trial_start', values='conversion_rate')

heatmap_data = heatmap_data.sort_index()

plt.figure(figsize=(12, 5))
ax = sns.heatmap(heatmap_data, annot=True, fmt=".1%", cmap="Blues", cbar_kws={'label': 'Conversion Rate'})

ax.set_yticklabels([d.strftime('%Y-%m-%d') for d in heatmap_data.index], rotation=0)

plt.title('Weekly Cohort Conversion Rate Heatmap')
plt.ylabel('Trial Start Week')
plt.xlabel('Weeks Since Trial Start')
plt.tight_layout()
plt.show()

In [ ]:
df_eda[['ts_start', 'trial_start', 'trial_end', 'converted_at']].agg(['min', 'max'])

### Observtions: 
- All converted organizations bought the subscriotion at least 2 weeks after trial start and first action
- Most of them did that within the last week of trial
- Around a half of converters did it after the trial end (30 days). Of those, most converted by day 45 since trial start.
- Some weeks were better than the others in terms of overall performance

#### Questions: 
- Does this mean that these organizations were consistently active during the first 2 weeks of trial?
- Was this conversion pattern consistent across install cohorts?

In [ ]:
# Filter converters
df_converted = df_eda[df_eda['converted'] == 1].copy()

# Calculate week numbers relative to trial start
df_converted['week_last_action'] = ((df_converted['last_action_ts'] - df_converted['trial_start']).dt.days // 7)+1
df_converted['weeks_to_convert_tr'] = (df_converted['days_to_convert_tr']// 7)+1


# Aggregate conversion info + last active trial week
converters = (
    df_converted
    .groupby('organization_id')
    .agg(
        weeks_to_convert=('weeks_to_convert_tr', 'min'),
        last_active_trial_week=('week_last_action', 'max')
    )
    .reset_index()
)

# Heatmap: days_to_convert vs last_active_trial_week
heatmap = (
    converters
    .groupby(['weeks_to_convert', 'last_active_trial_week'])
    .size()
    .unstack(fill_value=0)
    .sort_index()
    .sort_index(axis=1)
)
heatmap_norm = heatmap / heatmap.values.sum()
annotations = heatmap_norm.applymap(lambda x: f"{x:.1%}" if x > 0 else "")

plt.figure(figsize=(10, 6))
sns.heatmap(
    heatmap_norm,
    cmap='viridis',
    linewidths=0.5,
    linecolor='gray',
    annot=annotations,
    fmt="",
    annot_kws={"size": 8}, 
    cbar_kws = {
    'label': 'Share of Total Converters',
    'format': mticker.PercentFormatter(xmax=1.0)
    }
)
plt.title('Weeks to Convert vs Last Active Trial Week')
plt.xlabel('Last Active Trial Week')
plt.ylabel('Weeks to Convert from Trial Start')
plt.gca().invert_yaxis() 
plt.tight_layout()
plt.show()

#### Observtions & Questions: 
- Absolute majority of converters is only active on week 1 (0-6 days since trial start), but convert around the time of trial end.
- What is weekly retention like for convertes and non-converters?

In [ ]:
from plotly.subplots import make_subplots

df_viz = df_eda.copy()

# Step 1: DAILY retention
daily_counts = (
    df_viz
    .groupby(['days_since_first_action', 'converted'])['organization_id']
    .nunique()
    .reset_index(name='active_orgs')
)

daily_counts['retention'] = daily_counts.groupby('converted')['active_orgs'].transform(lambda x: x / x.max())
daily_counts['interval'] = 'Daily'
daily_counts = daily_counts.rename(columns={'days_since_first_action': 'period'})

# Step 2: WEEKLY retention
df_viz['weeks_since_first_action'] = (df_viz['days_since_first_action'] // 7).astype(int)

weekly_counts = (
    df_viz
    .groupby(['weeks_since_first_action', 'converted'])['organization_id']
    .nunique()
    .reset_index(name='active_orgs')
)

weekly_counts['retention'] = weekly_counts.groupby('converted')['active_orgs'].transform(lambda x: x / x.max())
weekly_counts['interval'] = 'Weekly'
weekly_counts = weekly_counts.rename(columns={'weeks_since_first_action': 'period'})

# Combine both for easy plotting
combined = pd.concat([daily_counts, weekly_counts], ignore_index=True)

# Plot side-by-side using Plotly subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Daily Retention", "Weekly Retention"),
    shared_yaxes=True
)

# Add traces for each interval
for converted_val in combined['converted'].unique():
    # Daily traces
    daily_data = combined[(combined['converted'] == converted_val) & (combined['interval'] == 'Daily')]
    fig.add_trace(
        go.Scatter(
            x=daily_data['period'],
            y=daily_data['retention'],
            mode='lines+markers',
            name=f'Daily - Converted {converted_val}',
        ),
        row=1, col=1
    )
    
    # Weekly traces
    weekly_data = combined[(combined['converted'] == converted_val) & (combined['interval'] == 'Weekly')]
    fig.add_trace(
        go.Scatter(
            x=weekly_data['period'],
            y=weekly_data['retention'],
            mode='lines+markers',
            name=f'Weekly - Converted {converted_val}',
        ),
        row=1, col=2
    )

# Update layout
fig.update_layout(
    title_text="Daily vs Weekly Retention by Conversion Status",
    height=400,
    width=1000,
    legend_title="Group",
    xaxis_title="Days Since First Action",
    xaxis2_title="Weeks Since First Action",
    yaxis_title="Normalized Retention",
)

fig.show()

In [ ]:
df_viz = df_eda.copy()
df_viz = df_viz[df_eda['active_span_days'] > 1]

org_activity = (
    df_viz
    .groupby(['organization_id', 'converted'])
    .agg(
        active_days=('active_days', 'max'),
        active_weeks=('active_weeks', 'first')
    )
    .reset_index()
)

org_activity['days_per_week'] = org_activity['active_days'] / org_activity['active_weeks']

fig = px.box(
    org_activity,
    x='converted',
    y='days_per_week',
    title='Distribution of Active Days per Active Week by Conversion Status',
    labels={'converted': 'Converted', 'days_per_week': 'Active Weeks / Active Days'}
)

fig.show()

#### Observtions: 
- Daily retention: 80% of both converters and non-converters churn after day1
- After Day 1 both daily and weekly retention is stable, starting from week 2 Converted orgs have slightly higher retention
- Converted organizations have higher weekly engagement density (2.45 vs 2 days) and lower weekly engagement variance

#### Observations & Questions
- Converted organizations with activity span between 15 and 28 days had fewer total active days on average compared to not converters (i.e. lower engagement density)
- Oterwise, there is no noticeable difference

#### Observations
- Converted organizations with activity span between 15 and 28 days had fewer total active days on average compared to not converters (i.e. lower engagement density)
- Oterwise, there is no noticeable difference

### Engagement rates per activity

In [ ]:
# Since all the conversions happen 14+ days since first action, I will use the first 2 weeks as a prediction dataset to normalise the data

#df_cap = df_eda[df_eda['days_since_first_action'] < 14].reset_index(drop=True)
df_cap = df_eda[df_eda['ts_start'] <= df_eda['converted_at']].reset_index(drop=True)
df_cap.head(3)

#### Identifying activities with the highest engagement

In [ ]:
def summarize_engagement_by_category(
    df,
    category_col,                # e.g. 'activity_name' or 'employee_act'
    user_group_col='user_group',
    org_col='organization_id',
    time_col='ts_start'
):
    # Total records and orgs engaged
    summary_table = (
        df
        .groupby([category_col, user_group_col])
        .agg(
            total_records=(time_col, 'count'),
            orgs_engaged=(org_col, 'nunique')
        )
        .reset_index()
    )

    # Pivot tables
    pivot_total_records = summary_table.pivot(
        index=category_col,
        columns=user_group_col,
        values='total_records'
    ).fillna(0).astype(int)

    pivot_orgs_engaged = summary_table.pivot(
        index=category_col,
        columns=user_group_col,
        values='orgs_engaged'
    ).fillna(0).astype(int)

    merged_pivot = pd.concat(
        [pivot_total_records, pivot_orgs_engaged],
        axis=1,
        keys=['total_records', 'orgs_engaged']
    )

    # Engagement rate
    
    total_orgs_per_group = df.groupby(user_group_col)[org_col].nunique().to_dict()

    engagement_rate = merged_pivot['orgs_engaged'].copy()
    for col in engagement_rate.columns:
        engagement_rate[col] = engagement_rate[col] / total_orgs_per_group.get(col, 1)

    engagement_rate.columns = pd.MultiIndex.from_product(
        [['engagement_rate'], engagement_rate.columns]
    )

    # Merge everything
    merged_with_engagement = pd.concat([merged_pivot, engagement_rate], axis=1)

    # Add "overall" columns for sorting
    merged_with_engagement[('total_records', 'overall')] = merged_with_engagement['total_records'].sum(axis=1)
    merged_with_engagement[('orgs_engaged', 'overall')] = merged_with_engagement['orgs_engaged'].sum(axis=1)
    merged_with_engagement[('engagement_rate', 'overall')] = (
        merged_with_engagement[('orgs_engaged', 'overall')] /
        sum(total_orgs_per_group.values())
    )

    merged_with_engagement = merged_with_engagement.sort_values(
        by=('engagement_rate', 'overall'),
        ascending=False
    )

    merged_with_engagement = merged_with_engagement.drop(
        columns=[
            ('total_records', 'overall'),
            ('orgs_engaged', 'overall'),
            ('engagement_rate', 'overall')
        ]
    )

    return merged_with_engagement

In [ ]:
#Setting the treshold for the most relevant activities

engagement = summarize_engagement_by_category(df_cap, category_col='activity_name')

# Filtering the events with at least 5% engagement rate in any user group
top_engagement = engagement[
    (engagement[('engagement_rate', 0)] > 0.02) |
    (engagement[('engagement_rate', 1)] > 0.02) |
    (engagement[('engagement_rate', 2)] > 0.02) 
]

styled_engagement = (
    top_engagement
    .reset_index()
    .style
    .background_gradient(subset=['engagement_rate'], cmap='viridis')
    .format({
        ('engagement_rate', 0): '{:.1%}',
        ('engagement_rate', 1): '{:.1%}',
        ('engagement_rate', 2): '{:.1%}'
    })
    .set_caption("Engagement rate per activity type")
)
styled_engagement

#### Observations & Questions:

-  _Scheduling.Shift.Created_  engagement is around 90% for both converters and non-converters. Why do organizations convert if they never scheduled a shift?
- _Mobile.Schedule.Loaded_  is used by only about 50% of organizations. Does this mean they don't have employees? Or they use it in admin-only mode?
- _PunchClock.PunchedOut_	is not in top 10 by engagement and not even close to  _PunchClock.PunchedIn_ - do they punch out automatically?
- _Scheduling.Shift.AssignmentChanged_, _PunchClock.PunchedIn_ and _Scheduling.Shift.Approved_ activities are very different for Group 1 (converters during trial) and could be good predictors.
- _Scheduling.Template.ApplyModal.Applied_ is much higher for Group 1 (converters after trial)

In [ ]:
def engagement_summary_no_activity(df, exclude_activities=None, mode='or'):
    """
    Generate an engagement summary for organizations that did NOT use specified activity/activities.

    Parameters:
    - df (pd.DataFrame): Source dataframe with required columns
    - exclude_activities (str or list of str): Activities to exclude by name
    - mode (str): 'or' (default) excludes orgs with any of the activities;
                  'and' excludes only those with all the activities

    Returns:
    - pd.Styler: Styled engagement summary
    """

    df = df.copy()

    # Normalize exclude_activities input
    if exclude_activities is None:
        exclude_activities = []
    elif isinstance(exclude_activities, str):
        exclude_activities = [exclude_activities]

    # Find organizations to exclude
    if mode == 'or':
        # Exclude orgs that used ANY of the activities
        orgs_with_excluded = df[df['activity_name'].isin(exclude_activities)]['organization_id'].unique()

    elif mode == 'and':
        # Find orgs that used ALL of the activities
        activity_per_org = df[df['activity_name'].isin(exclude_activities)].groupby('organization_id')['activity_name'].nunique()
        orgs_with_excluded = activity_per_org[activity_per_org == len(set(exclude_activities))].index

    else:
        raise ValueError("mode must be either 'or' or 'and'")

    # Filter out excluded orgs
    df_filtered = df[~df['organization_id'].isin(orgs_with_excluded)].sort_values(['organization_id', 'ts_start']).reset_index(drop=True)

    # Count remaining organizations
    org_count = df_filtered['organization_id'].nunique()

    # Summarize engagement
    engagement = summarize_engagement_by_category(df_filtered, category_col='activity_name')

    excluded_label = ', '.join(exclude_activities) if exclude_activities else "None"
    logic_label = "any of" if mode == 'or' else "all of"

    # Styled result
    styled = (
        engagement
        .reset_index()
        .style
        .background_gradient(subset=['engagement_rate'], cmap='viridis')
        .format({
            ('engagement_rate', 0): '{:.1%}',
            ('engagement_rate', 1): '{:.1%}',
            ('engagement_rate', 2): '{:.1%}'
        })
        .set_caption(
            f"<b><span style='font-size:16px'>Engagement "
            f"excluding orgs that used {logic_label}:</b> <br/>{excluded_label} . <br/> <b> Total: {org_count} organizations</span></b>"
        )
    )

    return styled

In [ ]:
#no_shifts = engagement_summary_no_activity(df_cap, exclude_activities=['A', 'B'], mode='or')

no_shifts = engagement_summary_no_activity(df_cap, exclude_activities=['Scheduling.Shift.Created'], mode='or')
no_shifts

In [ ]:
# Define the list of activities to exclude
#employee_activities = ['Mobile.Schedule.Loaded', 'PunchClock.PunchedIn', 'Absence.Request.Created', 'Scheduling.Availability.Set', 'Scheduling.OpenShiftRequest.Created']
employee_activities = ['Mobile.Schedule.Loaded']

# Then pass it to your function
no_employee = engagement_summary_no_activity(df_cap, exclude_activities=employee_activities, mode='or')
no_employee

In [ ]:
no_shifts_mobile = engagement_summary_no_activity(df_cap, exclude_activities=['Scheduling.Shift.Created', 'Mobile.Schedule.Loaded'], mode='or')
no_shifts_mobile

#### Observations and assumptions: 
- The organizations that didn't engage in sheduling shifts, mainly engaged in viewing schedule on mobile and didn't engage in any meaningful activity involving scheduling, time tracking or accounting.
- Around 50% of total organizations (both converted and not) did not engage in any employee-driven activities. I assume that it is possible to use Planday in admin-only mode without explicit time tracking.
- Mobile.Schedule.Loaded has the biggest overlap with other emploee activities, so I will use it as a predictor for any employee activity.
- Almost all the organizations (except 14) engaged either in Scheduling.Shift.Created or Mobile.Schedule.Loaded.

In [ ]:
#Building Makrov chains to see which events happened right before churn 

df_chains = df_cap.sort_values(['organization_id', 'ts_start'])

# Adding 'churned' and 'converted' states

final_activities = df_cap[['organization_id', 'converted', 'converted_at']].drop_duplicates(subset=['organization_id'])
final_activities['activity_name'] = final_activities['converted'].apply(lambda x: 'Converted' if x == 1 else 'Churned')
final_activities['ts_start'] = final_activities['converted_at']
final_activities = final_activities[['organization_id', 'activity_name', 'ts_start']]

df_chains_all = pd.concat([df_chains[['organization_id', 'activity_name', 'ts_start']], final_activities], ignore_index=True)
df_chains_sorted = df_chains_all.sort_values(['organization_id', 'ts_start'])

import plotly.express as px

def plot_transitions(df, title):
    df['next_state'] = df.groupby('organization_id')['activity_name'].shift(-1)
    
    transitions = (
        df.dropna(subset=['next_state'])
        .groupby(['activity_name', 'next_state'])
        .size()
        .reset_index(name='count')
    )
    
    transition_matrix = transitions.pivot(index='activity_name', columns='next_state', values='count').fillna(0)
    transition_prob = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)
    
    cols_to_plot = [col for col in ['Converted', 'Churned'] if col in transition_prob.columns]
    if not cols_to_plot:
        print("No transitions to 'Converted' or 'Churned' found.")
        return
    
    final_transitions = transition_prob[cols_to_plot].copy()
    
    # Scale each column independently to [0,1]
    final_transitions_scaled = final_transitions.apply(
        lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() > x.min() else 0
    )
    
    # Sort by sum of scaled values for order
    final_transitions_scaled['sum'] = final_transitions_scaled.sum(axis=1)
    final_transitions_scaled = final_transitions_scaled.sort_values('sum', ascending=False)
    
    # Select top 10 activities only
    final_transitions_scaled_top10 = final_transitions_scaled.head(10).drop(columns='sum')
    
    # Melt for plotly express
    plot_df = final_transitions_scaled_top10.reset_index().melt(
        id_vars='activity_name', value_vars=cols_to_plot,
        var_name='Transition To', value_name='Scaled Probability'
    )
    
    fig = px.bar(
    plot_df,
    y='activity_name',
    x='Scaled Probability',
    color='Transition To',
    orientation='h',
    barmode='group',
    title=title,
    labels={'activity_name': 'Current State', 'Scaled Probability': 'Scaled Transition Probability'},
    color_discrete_map={
        'Converted': 'orange',
        'Churned': 'steelblue'
        }
    )
    
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.show() 
    
plot_transitions(df_chains_sorted, "Scaled Transitions to 'Converted' and 'Churned'")

#### Observations

Some activities have 
- Scheduling.Shift.Created and Mobile.Schedule.Loaded  - these are core functions of the app and they have the highest engagement rate, so they alone are unlikely to cause conversion or churn
- Scheduling.Shift.Approved, Scheduling.Shift.AssignementChanged and Absence.Request.Approved have the highest difference between converted and churned groups
- Timesheets.BulkApprove.Confirmed only leads to churn

## Feature engineering and prediction

In [ ]:
#pd.set_option('display.max_rows', 100)
df_fe = df_cap.copy()
df_fe

In [ ]:
df_dim = (
    df_fe
    .groupby(['organization_id', 'converted'])
    .agg(
        days_to_action =('days_to_action', 'first'),
        first_action_weekday=('first_action_weekday', 'first'),
        first_action_hour=('first_action_hour', 'first'),
        active_span_days =('active_span_days', 'first'),
        active_days =('active_days', 'first'),
        active_weeks =('active_weeks', 'first'),
        activity_density =('activity_density', 'first')
    )
    .sort_values(['organization_id', 'converted'])
    .reset_index()
)
#df_dim['first_action_week'] = df_fe['first_action_ts'].dt.isocalendar().week.astype(int)

d1_flag = df_fe[df_fe['days_since_first_action'] == 1] \
            .groupby('organization_id').size() \
            .to_frame('d1_eng')

d1_flag['d1_eng'] = 1
df_dim = df_dim.merge(d1_flag[['d1_eng']], on='organization_id', how='left')
df_dim['d1_eng'] = df_dim['d1_eng'].fillna(0).astype(int)

df_dim

In [ ]:
def calculate_custom_metrics(df_eda, day_start=0, day_end=9999):
    # Step 1: Filter by days
    df_filtered = df_eda[
        (df_eda['days_since_first_action'] >= day_start) &
        (df_eda['days_since_first_action'] <= day_end)
    ].copy()


    all_orgs = df_filtered['organization_id'].unique()

    # Get max days per org to normalise the counts
    max_days = df_filtered.groupby('organization_id')['days_since_first_action'].max().replace(0, 1)

    # Top activities with at least 3% engagement
    activities = [
    'Scheduling.Shift.Created',
    'Mobile.Schedule.Loaded',
    'Scheduling.Shift.AssignmentChanged',
    #'PunchClock.PunchedIn',
    #'PunchClock.PunchedOut',
    'Scheduling.Shift.Approved',
    'Communication.Message.Created',
    'Scheduling.Template.ApplyModal.Applied',
    'Scheduling.Availability.Set',
    'Absence.Request.Created',
    'Absence.Request.Approved',
    'Timesheets.BulkApprove.Confirmed'
    ]

    agg = df_filtered.groupby(['organization_id', 'activity_name', 'bulk'], dropna=False).agg(
        total_records=('records', 'sum'),
        total_events=('events', 'sum')
    ).reset_index()

    # Step 6: Pivot for easier metric access
    records_pivot = agg.pivot_table(index=['organization_id', 'activity_name'], columns='bulk',
                                    values='total_records', fill_value=0)
    events_pivot = agg.pivot_table(index=['organization_id', 'activity_name'], columns='bulk',
                                   values='total_events', fill_value=0)

    # Step 7: Total events per org per activity
    total_events_per_activity = agg.groupby(['organization_id', 'activity_name'])['total_events'].sum().unstack(fill_value=0)

    # Step 8: Prepare index of orgs
    orgs = pd.Index(all_orgs, name='organization_id')

    # Step 9: Helpers
    def get_records(activity, bulk=None):
        try:
            if bulk is None:
                return records_pivot.xs(activity, level='activity_name').sum(axis=1).reindex(orgs, fill_value=0)
            return records_pivot.loc[(slice(None), activity), bulk].droplevel('activity_name').reindex(orgs, fill_value=0)
        except KeyError:
            return pd.Series(0, index=orgs)

    def get_events(activity, bulk=None):
        try:
            if bulk is None:
                return events_pivot.xs(activity, level='activity_name').sum(axis=1).reindex(orgs, fill_value=0)
            return events_pivot.loc[(slice(None), activity), bulk].droplevel('activity_name').reindex(orgs, fill_value=0)
        except KeyError:
            return pd.Series(0, index=orgs)

    # Step 10: Compute metrics
    df_metrics = pd.DataFrame(index=orgs)

    df_metrics['template_usage'] = get_events('Scheduling.Template.ApplyModal.Applied') / get_events('Scheduling.Shift.Created')
    df_metrics['availability_to_shifts'] = get_events('Scheduling.Availability.Set') / get_events('Scheduling.Shift.Created')
    df_metrics['views_to_shifts'] = get_records('Mobile.Schedule.Loaded') / get_records('Scheduling.Shift.Created')
    df_metrics['punch_in_views'] = get_records('PunchClock.PunchedIn') / get_records('Mobile.Schedule.Loaded')
    #df_metrics['punch_in_out'] = get_records('PunchClock.PunchedOut') / get_records('PunchClock.PunchedIn')
    df_metrics['change_rate'] = get_records('Scheduling.Shift.AssignmentChanged') / get_records('Scheduling.Shift.Created')
    df_metrics['messages_shifts'] = get_events('Communication.Message.Created') / get_events('Scheduling.Shift.Created')
    df_metrics['messages_views'] = get_records('Scheduling.Shift.AssignmentChanged') / get_records('Mobile.Schedule.Loaded')
    df_metrics['approve_rate'] = get_records('Scheduling.Shift.Approved') / get_records('Scheduling.Shift.Created')

    # Step 11: Add normalized per-day total events
    for activity in activities:
        col_name = f'daily_{activity}'
        if activity in total_events_per_activity.columns:
            normalized = total_events_per_activity[activity].reindex(orgs, fill_value=0) / max_days.reindex(orgs, fill_value=1)
        else:
            normalized = pd.Series(0, index=orgs)
        df_metrics[col_name] = normalized
    
    # Sum records for each activity across all bulk types
    activity_records_sum = records_pivot.groupby(level=['organization_id', 'activity_name']).sum()

    # Step 12: Cleanup
    df_metrics.replace([float('inf'), -float('inf')], 0, inplace=True)
    df_metrics.fillna(0, inplace=True)

    return df_metrics.reset_index()

In [ ]:
#Testing on just 2 days of engagement
metrics_df = calculate_custom_metrics(df_fe, day_start=0, day_end=14)

'''
cols_to_drop = [
    col for col in metrics_df.select_dtypes(include='number').columns
    if not col.startswith('daily_') and col != 'converted'
]
metrics_df = metrics_df.drop(columns=cols_to_drop)
'''

metrics_df

In [ ]:
zero_var_cols = metrics_df.columns[metrics_df.nunique() <= 1]
print("Columns with zero or one unique value:", zero_var_cols.tolist())

In [ ]:
df_feat = df_dim.merge(metrics_df, on='organization_id', how='left')
df_feat.head()

In [ ]:
#print(df_feat.isnull().sum())

In [ ]:
#print(df_feat.head())
print(df_feat.columns)

In [ ]:
#pip install scikit-learn imbalanced-learn xgboost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

df=df_feat.copy()

X = df.drop(columns=['converted', 'organization_id'])
y = df['converted']

#categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
#X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)  

# Now you can continue with scaling, modeling, etc.
scaler = StandardScaler()
#X_scaled = pd.DataFrame(scaler.fit_transform(X_encoded), columns=X_encoded.columns)
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)


# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# Apply SMOTE on training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Define models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'KNN': KNeighborsClassifier()
}

# Train, predict, and evaluate
for name, model in models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train_res, y_train_res)
    
    y_pred = model.predict(X_test)
    y_proba = None
    # For AUROC, need predicted probabilities for the positive class
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):  # fallback
        y_proba = model.decision_function(X_test)
    
    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")
    
    # Detailed classification report
    print("Classification report:")
    print(classification_report(y_test, y_pred, digits=4))
    
    # AUROC
    if y_proba is not None:
        auroc = roc_auc_score(y_test, y_proba)
        print(f"AUROC: {auroc:.4f}")
    else:
        print("AUROC: N/A (no probability estimates available)")

#### Prediction result

XGBoost model has shown the best accuracy overall. Although more feature engineering and pruning to improve Converters prediction, we can look at the most important features.

In [ ]:
import shap

# Fit your model (already done)
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_res, y_train_res)

# Create SHAP explainer for tree-based models
explainer = shap.Explainer(xgb_model, X_train_res)

# Calculate SHAP values for the training set
shap_values = explainer(X_train_res)

In [ ]:
shap.summary_plot(shap_values, X_train_res, max_display=30)

#### Features
According to the model, the most important features. Here are the ones that had a clear positive or negative contribution:

Independent metrics:

- _Days to action_: organizations that start using the app on the day of trial start are more likely to convert

Note: daily metrics were capped (only first 3 days), the above are historic

App metrics:

- Organizations with high _Shift assignement change rate_ are less likely to convert
- Organizations with higher _Punch in to View_ ratio are more likely to convert
- Organizations with higher _Template usage_ are more likely to convert
- Organizations with low ratio of _Views to Shifts_ scheduled are more likely to convert -> this could indicate that actively run a business and already have real employees
- Organizations with high number of _Messages Created_ scheduled are less likely to convert -> this could indicate that the messaging tool is not convenient, or that users don't understaand how to use other fun

Punch in to punch out rate is quite low and often 0, so I assume that this is an optional action and employees are punched out automatically when their shift ends.

### Trial goals

Based on the model and EDA results, I am going to monitor these trial goals:

Core functionality:
Scheduling.Shift.Created -> Mobile.Schedule.Loaded -> Scheduling.Shift.Approved

Admin functionality:
Scheduling.Template.ApplyModal.Applied -> Scheduling.Shift.Created ->  Scheduling.Shift.AssignmentChanged -> Scheduling.Shift.Approved

Since there is no hard sequence of events, I ordered them in the order or the most intuitive discovery in my opinion.

# Part 2

### 2.1 Trial goals

In [ ]:
# Defining the same table as made in SQL
# I will take df_raw as the source

funnel_steps = {
    'Scheduling.Shift.Created': 1,
    'Mobile.Schedule.Loaded': 2,
    'Scheduling.Shift.AssignmentChanged': 3,
    'Scheduling.Shift.Approved': 4,
    'Scheduling.Template.ApplyModal.Applied': 5
}

organizations =  (
    df_raw
    .groupby(['organization_id'])
    .agg(
        trial_start=('trial_start', 'min'),
        trial_start_real=('timestamp', 'min'),
        trial_end=('trial_end', 'max')
    )
    .sort_values(['organization_id'])
    .reset_index()
)

organizations['trial_week'] = organizations['trial_start'].dt.to_period('W').apply(lambda r: r.start_time)

In [ ]:
trial_goals = df_raw[df_raw['activity_name'].isin(funnel_steps.keys())].copy()

# Map funnel steps
trial_goals['funnel_step'] = trial_goals['activity_name'].map(funnel_steps)

# Keep only required columns
trial_goals = trial_goals[[
    'timestamp',
    'organization_id',
    'funnel_step',
    'activity_name'
]]

trial_goals_mart=trial_goals.merge(organizations, on='organization_id', how='inner')
trial_goals_mart

In [ ]:
funnel_counts = (
    trial_goals_mart
    .groupby(['activity_name', 'funnel_step'])['organization_id']
    .nunique()
    .reset_index(name='orgs')
)

funnel_counts = funnel_counts.sort_values('funnel_step')

first_step_count = funnel_counts.loc[funnel_counts['funnel_step'] == 1, 'orgs'].values[0]
funnel_counts['share'] = funnel_counts['orgs'] / first_step_count
funnel_counts['label'] = funnel_counts['orgs'].astype(str) + ' orgs (' + (funnel_counts['share'] * 100).round(1).astype(str) + '%)'

fig = px.funnel(
    funnel_counts,
    y='activity_name',
    x='orgs',
    labels={'orgs': 'Organizations', 'activity_name': 'Funnel Step'},
    title='Trial Goals Completed by Organizations'
)

fig.update_layout(
    yaxis=dict(
        categoryorder='array',
        categoryarray=funnel_counts.sort_values('funnel_step')['activity_name'].tolist()
    )
)

for i, row in funnel_counts.iterrows():
    fig.add_annotation(
        x=row['orgs'],
        y=row['activity_name'],
        text=row['label'],
        showarrow=False,
        xanchor='left',
        yanchor='middle',
        font=dict(size=12, color='black')
    )

fig.show()

#### Observations
Completion rate of the least popular trial goal is 12,7%, but since they are not strictly locked on one another, total completion rate could be lower.
Also, this is all data for all the trial cohorts, so some organizations didn't have enough time to complete them all.

In [ ]:
# Heatmap

funnel_heatmap_data = (
    trial_goals_mart
    .groupby(['trial_week', 'activity_name'])['organization_id']
    .nunique()
    .reset_index(name='orgs_completed')
)

weekly_orgs = (
    organizations
    .groupby('trial_week')['organization_id']
    .nunique()
    .reset_index(name='total_orgs')
)

funnel_heatmap_data = funnel_heatmap_data.merge(weekly_orgs, on='trial_week')
funnel_heatmap_data['completion_rate'] = funnel_heatmap_data['orgs_completed'] / funnel_heatmap_data['total_orgs']

fig = px.imshow(
    funnel_heatmap_data.pivot(index='activity_name', columns='trial_week', values='completion_rate'),
    labels=dict(x="Trial Start Week", y="Funnel Step (Activity)", color="Completion Rate"),
    color_continuous_scale='Blues',
    title="Funnel Completion Rate by Trial Start Week",
    aspect="auto"
)

fig.update_layout(
    xaxis_title="Trial Start Week",
    yaxis_title="Activity",
    yaxis=dict(
        categoryorder='array',
        categoryarray=list(funnel_steps.keys())  # Optional: enforce funnel order
    )
)

fig.show()

#### Observations
Trial goal copmletion per weekly cohort varies from 8% to 21% depending on trial start date

### 2.2 Trial activation

In [ ]:
# Trial activation summary per organization
trial_activation = (
    trial_goals_mart
    .groupby('organization_id')
    .agg(
        max_step_ts=('timestamp', 'max'),
        max_funnel_step=('funnel_step', 'max'),
        total_steps=('activity_name', 'nunique')
    )
    .reset_index()
)

# Determine trial_status
trial_activation['trial_status'] = trial_activation['total_steps'].apply(
    lambda x: 'Completed' if x > 4 else 'Not Completed'
)
trial_activation

In [ ]:
trial_activation_mart = trial_activation.merge(
    organizations,
    on='organization_id',
    how='inner'
)

trial_activation_mart['days_since_trial_start'] = (
    (trial_activation_mart['max_step_ts'] - trial_activation_mart['trial_start']).dt.days
)
trial_activation_mart['days_since_trial_start_real'] = (
    (trial_activation_mart['max_step_ts'] - trial_activation_mart['trial_start_real']).dt.days
)

funnel_step_reverse = {v: k for k, v in funnel_steps.items()}
trial_activation_mart['last_trial_activity'] = trial_activation_mart['max_funnel_step'].map(funnel_step_reverse)

trial_activation_mart.rename(columns={
    'max_step_ts': 'updated_at',
    'max_funnel_step': 'last_trial_step'
}, inplace=True)

trial_activation_mart = trial_activation_mart[[
    'organization_id',
    'updated_at',
    'days_since_trial_start_real',
    'days_since_trial_start',
    'last_trial_step',
    'last_trial_activity',
    'total_steps',
    'trial_status',
    'trial_start',
    'trial_start_real',
    'trial_end',
    'trial_week'
]]

trial_activation_mart

In [ ]:
status_heatmap = (
    trial_activation_mart
    .groupby(['trial_week', 'trial_status'])
    .size()
    .reset_index(name='org_count')
)

total_per_week = (
    trial_activation_mart
    .groupby('trial_week')['organization_id']
    .nunique()
    .reset_index(name='total_orgs')
)

status_heatmap = status_heatmap.merge(total_per_week, on='trial_week')
status_heatmap['percent'] = (status_heatmap['org_count'] / status_heatmap['total_orgs']) * 100
status_heatmap['label'] = status_heatmap.apply(
    lambda row: f"{row['percent']:.1f}%", axis=1
)

z_values = status_heatmap.pivot(index='trial_status', columns='trial_week', values='org_count').fillna(0)
text_labels = status_heatmap.pivot(index='trial_status', columns='trial_week', values='label').fillna("")

ordered_status = ['Completed', 'Not Completed', 'Unknown']
z_values = z_values.reindex([s for s in ordered_status if s in z_values.index])
text_labels = text_labels.reindex(z_values.index)

fig = go.Figure(data=go.Heatmap(
    z=z_values.values,
    x=z_values.columns.astype(str),
    y=z_values.index,
    text=text_labels.values,
    texttemplate="%{text}",
    colorscale='Blues',
    colorbar_title="Organizations"
))

fig.update_layout(
    title="Trial Status Distribution by Trial Week",
    xaxis_title="Trial Week",
    yaxis_title="Trial Status"
)

fig.show()

#### Observations
Overall trial activation rate based on completing **all** trial goals is between 2.4% and 10% 

## Saving mart_ tables

In [ ]:
trial_goals_mart.to_csv('trial_goals_mart.csv', index=False)
trial_activation_mart.to_csv('trial_activation_mart.csv', index=False)